<a href="https://colab.research.google.com/github/hiroto-noguchi/useful_materials_for_research/blob/main/csv_to_textgrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  使用手順
  1. すべてのセルを実行する。
  1. ダイアログからmm:ss形式の時刻（start, end, label列を含む）を持つCSVファイルをアップロードする。

In [ ]:
!pip install tgt


In [ ]:
import pandas as pd
import os
from google.colab import files
import tgt

def mmss_to_seconds(time_val):
    """
    "mm:ss" 形式の文字列を秒数（float）に変換する関数です。
    0落ち（例："1:30" または "01:30"）にも対応します。
    例: "1:30" -> 1*60 + 30 = 90秒
    """
    # 万が一数値になっていたら文字列に変換
    if not isinstance(time_val, str):
        time_val = str(time_val)
    parts = time_val.strip().split(":")
    if len(parts) != 2:
        raise ValueError(f"時刻フォーマットが正しくありません: {time_val}")
    minutes, seconds = parts
    return float(minutes) * 60 + float(seconds)

def create_textgrid_from_dataframe(df, output_file, tier_name="Tier1"):
    """
    CSVファイルから読み込んだ DataFrame（'start', 'end', 'label' 列があることを前提）
    から、TextGrid ファイルを作成する関数です。
    もし最初の開始時刻が 0:00 でなければ、0:00～最初の開始時刻までの空白区間を追加します。
    """
    # 文字列から秒数に変換
    df["start"] = df["start"].apply(mmss_to_seconds)
    df["end"]   = df["end"].apply(mmss_to_seconds)

    # 開始時刻でソート
    df = df.sort_values(by="start")

    # CSV上の最初の開始時刻と全体の終了時刻を取得
    csv_global_start = df["start"].min()
    global_end = df["end"].max()

    # TextGridの全体開始時刻を0にするか、csvの最小値にするか判断
    tier_start = 0 if csv_global_start > 0 else csv_global_start

    # TextGrid オブジェクト生成
    tg = tgt.TextGrid()
    # IntervalTier の引数の順番は (start_time, end_time, name)
    tier = tgt.IntervalTier(tier_start, global_end, tier_name)

    # もしCSVの最初の開始時刻が0でなければ、0～csv_global_startまでの空白区間を追加
    if csv_global_start > 0:
        blank_interval = tgt.Interval(tier_start, csv_global_start, "")
        tier.add_interval(blank_interval)

    # CSVの各行のデータから Interval を作成して Tier に追加
    for row in df.itertuples(index=False):
        # 行ごとに start <= end になっているかチェック
        if row.start > row.end:
            print(f"Warning: 行の start {row.start} が end {row.end} より大きいためスキップします。")
            continue
        interval = tgt.Interval(row.start, row.end, row.label)
        tier.add_interval(interval)

    tg.add_tier(tier)
    tgt.write_to_file(tg, output_file, format="short")
    print(f"TextGridファイルを作成しました: {output_file}")

# CSV ファイルのアップロード
uploaded = files.upload()

if uploaded:
    # アップロードされた CSV ファイル名を取得
    csv_filename = list(uploaded.keys())[0]
    print(f"アップロードされたCSVファイル: {csv_filename}")

    # dtype=str とすることで、セル内容を文字列として読み込む
    df = pd.read_csv(csv_filename, dtype=str)

    # 前後の空白を除去
    df["start"] = df["start"].str.strip()
    df["end"] = df["end"].str.strip()
    df["label"] = df["label"].str.strip()

    # 入力ファイル名の拡張子だけを変更して出力ファイル名を生成（例: sample.csv -> sample.TextGrid）
    base_name = os.path.splitext(csv_filename)[0]
    output_filename = f"{base_name}.TextGrid"

    create_textgrid_from_dataframe(df, output_filename)
    files.download(output_filename)
else:
    print("ファイルがアップロードされていません。")
